In [ ]:
import os

### Loading tavily stuffs
from langchain.tools.tavily_search import TavilySearchResults 

### Loading langchain stuffs
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

### some more shitty stuffs
import requests
from bs4 import BeautifulSoup
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

import warnings
warnings.filterwarnings('ignore') # avoid warning messages importing packages

import json

REQUEST_PER_QUESTION = 2

In [ ]:
from dotenv import load_dotenv
load_dotenv()

# Tavily API Key
auth_key_tavily = os.environ.get("TAVILY_API_KEY")

# HuggingFace API Key
auth_key_hf = os.environ.get("HF_KEY")

# Langchain key
langchain_key = os.environ.get("LANGCHAIN_API_KEY")

In [ ]:
from huggingface_hub import snapshot_download
from pathlib import Path

# snapshot_download(repo_id="mistralai/Mistral-7B-Instruct-v0.3", allow_patterns=["params.json", "consolidated.safetensors", "tokenizer.model.v3"], local_dir='./Mistral_7B_Instruct_v0.3/')

In [ ]:
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.messages import UserMessage, AssistantMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest

import torch
from transformers import AutoModelForCausalLM
from transformers import BitsAndBytesConfig, pipeline

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

mistral_models_path = './Mistral_7B_Instruct_v0.3/' 

tokenizer = MistralTokenizer.from_file(f"{mistral_models_path}/tokenizer.model.v3")
# model = Transformer.from_file({mistral_models_path})
model = AutoModelForCausalLM.from_pretrained('mistralai/Mistral-7B-Instruct-v0.3', quantization_config=quantization_config)

completion_request = ChatCompletionRequest(messages=[UserMessage(content="Explain Machine Learning to me in a nutshell.")])
tokens = tokenizer.encode_chat_completion(completion_request).tokens
generated_ids = model.generate(torch.tensor([tokens]).to('cuda'), max_new_tokens=128)
result = tokenizer.decode(generated_ids[0].tolist())
print(result)

In [ ]:
# messages = [
#     {"role": "user", "content": "What is your favourite condiment?"},
#     {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
#     {"role": "user", "content": "Do you have mayonnaise recipes?"}
# ]
# 
# encodeds = tokenizer_auto.apply_chat_template(messages, return_tensors="pt")
# 
# generated_ids = model.generate(encodeds.to('cuda'), max_new_tokens=64, do_sample=True)
# decoded = tokenizer_auto.batch_decode(generated_ids)
# print(decoded[0])

In [ ]:
from langchain_huggingface import (
    HuggingFacePipeline, HuggingFaceEmbeddings, ChatHuggingFace, HuggingFaceEndpoint
)

from transformers import AutoTokenizer

tokenizer_auto = AutoTokenizer.from_pretrained('mistralai/Mistral-7B-Instruct-v0.3', max_length=128, skip_special_tokens=True)
tokenizer_auto.pad_token = tokenizer_auto.eos_token

pipe = pipeline(
    task="text-generation", 
    model=model, 
    tokenizer=tokenizer_auto, 
    do_sample=True, 
    max_new_tokens=128,
    truncation='only_first',
)

llm = HuggingFacePipeline(
    pipeline=pipe,
)

# llm = HuggingFaceEndpoint(
#     endpoint_url = "https://api-inference.huggingface.co/models/google/gemma-2b-it",
#     task='text-generation',
#     max_new_tokens=64,
#     top_k=10,
#     top_p=0.9,
#     repetition_penalty=1.03,
#     timeout=240,
#     huggingfacehub_api_token=auth_key_hf
# )

In [ ]:
llm('<s><INST>Your task is to make a ordered list of the items</INST> for items=(banana, apple), your output: 1. apple, 2. banana.</s><INST> Make list for items "eggs, bacon, toast"</INST>')

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

In [ ]:
from duckduckgo_search import DDGS

def web_search(query:str, num_results:int=2):
    results = DDGS().text(query, max_results=num_results)
    return [r["href"] for r in results] 

def scrape_text(url: str):
    # Send a GET request to the webpage
    try:
        response = requests.get(url)
        # Check if the request was successful
        if response.status_code == 200:
            # Parse the webpage using BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')
            # Extract the text from the webpage
            text = soup.get_text(separator=' ', strip=True)
            return text
        else:
            return f"Failed to retrieve the webpage. \
                    Status code: {response.status_code}"
    except Exception as e:
        return f"An error occurred: {e}"

In [ ]:
web_search("What is the impact of machine learning on audio faking?")

In [ ]:
# summary_template = f"use the {text} to summarize the question: {question}"

# def summary_template(question:str, text:str):
#     return f"""<s><INST>Answer the question:"{question}" by summarizing the text: "{text}" if the question cannot be answered using the text.</INST>"""

# summary_template = """<s><INST>Answer the question:"{question}" by summarizing the text: "{text}" if the question cannot be answered using the text.</INST>"""
# SUMMARY_PROMPT = ChatPromptTemplate.from_template(summary_template)

SUMMARY_PROMPT = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful AI bot.'),
    ('user', 'Answer the question:"{question}" by summarizing the text: "{text}". If the question cannot be answered using the text, return the text.' )
])

scrape_and_summarize_chain = RunnablePassthrough.assign(
    summary=RunnablePassthrough.assign(
        text=lambda x: scrape_text(x["url"])[:10]
    ) | SUMMARY_PROMPT | llm | StrOutputParser()
) | (lambda x: f"URL: {x['url']}\n\nSUMMARY: {x['summary']}")

In [ ]:
web_search_chain = (RunnablePassthrough.assign(
    urls = lambda x: web_search(x["question"])
) | (lambda x: [{"question": x["question"], "url": u} for u in x["urls"]]) 
  | scrape_and_summarize_chain.map())

In [ ]:
web_search_chain.invoke({"question": "What is the impact of machine learning on audio faking?"})

In [ ]:
import regex as re

In [ ]:
search_template = """
        <s>
        <INST>
            Write only 2 google search queries to search from the following question: "{question}". 
            Return a numbered list of the queries.
        </INST>
    """

SEARCH_PROMPT = ChatPromptTemplate.from_messages([
    ('user', 'Write only 2 google search queries to search from the following question: "{question}". Return a numbered list of the queries.' ),   
])

In [ ]:
search_question_chain = (SEARCH_PROMPT | llm | StrOutputParser() | (lambda x: re.findall(r'[1-9]\.\s?"(.*)"\n?', x)))

In [ ]:
search_qs_test = search_question_chain.invoke({'question': 'What is the impact of machine learning on audio faking?'})

In [ ]:
search_qs_test

In [ ]:
full_research_chain = search_question_chain |  (lambda x: [{"question": q} for q in x]) | web_search_chain.map()

In [ ]:
full_research_chain.invoke({'question': 'What is the impact of machine learning on audio faking?'})

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

In [ ]:
WRITER_SYSTEM_PROMPT = """
        <s>
        <INST>
            You are an AI critical thinker research assistant. 
            Your sole purpose is to write 
            well written, critically acclaimed, 
            objective and structured reports on given text or task. 
        </INST>
    """

# Report prompts from https://github.com/assafelovic/gpt-researcher/blob/master/gpt_researcher/master/prompts.py
RESEARCH_REPORT_TEMPLATE = """<s><INST>
    Using the information "{research_summary}", answer the following question or topic: "{question}" in a detailed report -- \
The report should focus on the answer to the question, should be well structured, informative, \
in depth, with facts and numbers if available and a minimum of 1,200 words.
You should strive to write the report as long as you can using all relevant and necessary information provided.
You must write the report with markdown syntax.
You MUST determine your own concrete and valid opinion based on the given information. Do NOT deter to general and meaningless conclusions.
Write all used source urls at the end of the report, and make sure to not add duplicated sources, but only one reference for each.
You must write the report in apa format.
Please do your best, this is very important to my career.</INST>"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", WRITER_SYSTEM_PROMPT),
        ("user", RESEARCH_REPORT_TEMPLATE),
    ]
)

In [ ]:
def collapse_list_of_lists(list_of_lists):
    content = []
    for l in list_of_lists:
        content.append("\n\n".join(l))
    return "\n\n".join(content)

In [ ]:
chain = (RunnablePassthrough.assign(
    research_summary=full_research_chain | collapse_list_of_lists
) | prompt | llm | StrOutputParser())

In [ ]:
chain.invoke({"question":'what is the impact of machine learning on audio faking'})